# INFORMATION RETRIEVAL (IR)

In [1]:
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import Data

In [3]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract TFIDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [11]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

C:\Users\Damianus Deni\Miniconda3\envs\jcopml\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# Document Similarity

In [12]:
from sklearn.metrics.pairwise import cosine_distances
# cosine_distance = 1 - cos_similarity

In [13]:
tfidf_matrix[0]

<1x548404 sparse matrix of type '<class 'numpy.float64'>'
	with 784 stored elements in Compressed Sparse Row format>

In [14]:
# satu dokumen dibandingkan dengan semua dokumen untuk mencari kemiripan
dist = cosine_distances(tfidf_matrix[0], tfidf_matrix)
dist

array([[0.        , 0.99141672, 0.98939957, ..., 0.99143713, 0.99322192,
        0.98486659]])

In [28]:
dist.argsort()
# mengurutkan peringkat dengan menampilkan index nya

array([[   0,  144,  215, ..., 1593, 1131,  932]], dtype=int64)

In [35]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [37]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [38]:
df.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

#### contoh penggunaan argsort 

In [23]:
import numpy as np

In [33]:
a = np.array([0, 10, 50, 4, 81])

In [34]:
a.argsort()

array([0, 3, 1, 2, 4], dtype=int64)

#### contoh penggunaan similarity dengan keyword

In [71]:
query = ["ginanjar"]
tfidf.transform(query)

<1x548404 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [72]:
def search(query, tfidf, tfidf_matrix, df):
    vec = tfidf.transform([query])
    dist = cosine_distances(vec, tfidf_matrix)
    best_idx = dist.argsort()[0, :10]
    return df.teks[best_idx] 

In [73]:
result = search ("""
Juru bicara pemerintah untuk penanganan virus corona Achmad Yurianto mengatakan, pemerintah mengkhawatirkan keberadaan orang tanpa gejala (OTG) yang menjadi sumber penularan Covid-19 di tengah masyarakat. "Kita masih tetap mewaspadai adanya penularan yang terjadi di lingkungan masyarakat. Ini disebabkan masih adanya kasus positif Covid-19 tanpa keluhan yang berada di tengah-tengah kita," ujar Yuri dalam konferensi pers di Graha BNPB, Sabtu (4/4/2020)

Artikel ini telah tayang di Kompas.com dengan judul "Jumlah Kasus Covid-19 Terus Bertambah, Pemerintah Khawatirkan Penularan dari OTG", https://nasional.kompas.com/read/2020/04/04/18382271/jumlah-kasus-covid-19-terus-bertambah-pemerintah-khawatirkan-penularan-dari.
Penulis : Dian Erika Nugraheny
Editor : Kristian Erdianto
""", tfidf, tfidf_matrix, df)

result

483     Wartawan Diculik Sindikat VCD Porno\n\nSalah s...
628     Menghindari Anthrax\nBERKAITAN dengan Idul Adh...
457     Menjawab Memorandum DPR\n\nDalam waktu dekat, ...
297     20 Tahun AIDS   Masih Banyak yang Belum Terjaw...
829     Presiden Bertemu Pimpinan TNI dan Polri\nPresi...
1580    Akan Dinaikkan hingga 700 Persen, PBB Vila di ...
638     Tasawuf dan Kebatinan/Kejawen\nOleh Abdurrahma...
1621    RI-GAM Sepakat Lanjutkan Dialog\n\nPemerintah ...
1520    Perusak Lingkungan Harus Diadili\n\nPihak-piha...
1584    Korban Banjir Dapat Ajukan "Class Action"\n\nP...
Name: teks, dtype: object

In [74]:
result.iloc[0]

'Wartawan Diculik Sindikat VCD Porno\n\nSalah seorang wartawan Harian Lantang yang terbit di Batam, Dian Budiana (22), disekap beberapa orang yang diduga berhubungan dengan sindikat pembuat VCD porno dengan bintang film lokal yang berusia muda. Sebab salah satu di antara penculik, memberitahu yang bersangkutan ditahan agar tidak menulis berita tentang VCD porno lagi.\n\nSejak tanggal 16-22 Februari 2001, harian Lantang menurunkan laporan berseri berbentuk investigasi yang ditulis oleh Dian Budiana bersama wartawan lainnya Taufik Hidayat. Namun tulisan terhenti sejak Dian tak muncul di kantor karena dikira pulang kampung tanpa izin.\n\nDian yang dalam laporannya di koran itu sering menggunakan nama samaran Iyank Sulanjani, baru diketahui diculik setelah masuk kantor pada hari Sabtu sekitar pukul 14.00 dalam keadaan lemas. Kehadiran Dian membuat kaget rekannya, karena mukanya penuh dengan darah mengering akibat luka pada kening dan darah yang mengalir dari dalam telinga kiri.\n\nKetika d

ini untuk menambah data baru pada bank data tfidf.transform(["""
teks tambahan"""])

# Keyword Extract

In [54]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [58]:
sorted_tfidf = tfidf_matrix[0].toarray().argsort()
sorted_tfidf

array([[274201, 365453, 365452, ..., 386361, 436633, 169209]], dtype=int64)

In [63]:
for idx in reversed(sorted_tfidf[0, -10:]):
    print(vocab[idx], tfidf_matrix[0].toarray()[0, idx])

ginandjar 0.5905638389928064
putusan 0.1978650873726769
penahanan 0.18989176211580433
hukum ginandjar 0.17318155723248
kuasa hukum 0.14299510558625728
rusman 0.13615815122356772
kejaksaan 0.13298116108647093
hakim rusman 0.13125826536502502
kuasa 0.12217625708688831
9 april 0.1178477353342584


In [64]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
    vec = tfidf.transform([doc])
    vocab = tfidf.get_feature_names()
    
    sorted_tfidf = vec.toarray().argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[0, -topk:])]

In [67]:
text2 = """
Amerikanisasi
di luar amerika serikat , amerikanisasi adalah istilah untuk menyebut pengaruh yang dimiliki amerika serikat di negara lain , contohnya budaya masyarakat , masakan , teknologi , praktik bisnis , atau teknik politiknya .
istilah ini sudah digunakan sejak tahun 1907 .
di amerika serikat sendiri , kata amerikanisasi berarti proses akulturasi adat dan nilai-nilai amerika serikat oleh imigran atau penduduk yang dianeksasi ( misalnya california ) .
hollywood ( industri film dan televisi amerika serikat ) mendominasi sebagian besar pasar media dunia .
inilah medium yang dipakai orang-orang di seluruh dunia untuk melihat gaya , adat , suasana , dan cara hidup amerika serikat .
beberapa perusahaan komputer terbesar di dunia juga berkantor pusat di amerika serikat , misalnya microsoft , apple , dell , dan ibm .
sebagian besar perangkat lunak yang dibeli di seluruh dunia juga diciptakan oleh perusahaan-perusahaan amerika serikat .
carayannis dan campbell menulis bahwa , " secara global , amerika serikat memiliki posisi yang sangat kuat di sektor perangkat lunak .
"
di jerman pada tahun 1920 - an , gerakan efisiensi amerika serikat disebut " rasionalisasi " .
ini adalah gerakan sosial dan ekonomi yang cukup kuat .
gerakan ini memandang secara berlebihan model - model amerika serikat , khususnya fordisme .
" rasionalisasi " berarti produktivitas yang lebih tinggi dan efisiensi yang lebih besar , menjanjikan bahwa ilmu pengetahuan akan memberi kemakmuran .
secara umum , rasionalisasi menjanjikan tingkatan modernitas baru dan diterapkan pada produksi dan konsumsi ekonomi dan administrasi publik .
berbagai versi rasionalisasi diperkenalkan oleh para industrialis dan demokrat sosial , insinyur dan arsitek , pendidik dan akademisi , feminis dan pekerja sosial kelas menengah , pejabat pemerintahan dan politikus partai .
sebagai ideologi dan praktik , rasionalisasi menantang dan mengubah tidak hanya mesin , pabrik , dan perusahaan bisnis , tetapi juga kehidupan warga jerman kelas menengah dan pekerja .
selama 15 tahun sejak 1950 sampai 1965 , investasi amerika serikat di eropa melonjak 800 % menjadi $ 13,9 miliar dan di komunitas ekonomi eropa naik 10 kali lipat menjadi $ 6,25 miliar .
pangsa eropa di dunia investasi amerika serikat naik dari 15 % ke 28 % .
investasi adalah model amerikanisasi yang paling jelas dan sering diperbincangkan .
meski begitu , investasi amerika di eropa hanya mewakili 5 % dari total investasi eropa dan perusahaan milik amerika di komunitas ekonomi eropa hanya mempekerjakan 2 atau 3 % total tenaga kerja .
alasan yang mendasari investasi amerika serikat ini bukan lagi biaya produksi rendah , pertumbuhan ekonomi cepat , atau laba tinggi di eropa , tetapi keinginan untuk mempertahankan posisi bersaing yang didasari superioritas teknologi amerika serikat .
penolakan investasi amerika serikat awalnya muncul di Perancis , kemudian menyebar ke negara-negara eropa lainnya .
opini publik mulai menolak iklan dan metode bisnis amerika , kebijakan personel , dan pemakaian bahasa inggris oleh perusahaan-perusahaan amerika .
kritik juga diarahkan ke sistem mata uang internasional yang dituduh sebagai penyebab kecenderungan inflasi akibat posisi dolar a.s. yang dominan .
akan tetapi pada 1970 - an , investasi eropa di a.s. meningkat lebih cepat daripada sebelumnya .
geir lundestad menemukan bahwa persepsi " amerika serikat membeli eropa " juga mulai berkurang .
berghahn ( 2010 ) menganalisis perdebatan mengenai manfaat konsep ' amerikanisasi ' dan ' westernisasi ' .
ia meninjau penelitian terkini mengenai hubungan eropa - amerika serikat selama perang dingin yang memengaruhi dampak budaya amerika serikat di eropa .
ia kemudian mendiskusikan penelitian yang relevan terhadap subjek ini di bidang sejarah ekonomi dan bisnis .
secara keseluruhan , artikelnya mencoba memaparkan bahwa orang-orang yang menerapkan konsep ' amerikanisasi ' ke penelitian mereka mengenai sejarah budaya dan / atau ekonomi sudah sadar tentang kerumitan hubungan lintas atlantik pada masa tersebut , apakah dipandang sebagai pertukaran dua arah atau proses sirkulasi .
"""

In [68]:
extract_keywords_tfidf(text2, tfidf)

['amerika',
 'amerika serikat',
 'serikat',
 'eropa',
 'investasi amerika',
 'rasionalisasi',
 'investasi',
 'ekonomi',
 'ekonomi eropa',
 'bisnis']

In [69]:
text3 = """Juru bicara pemerintah untuk penanganan virus corona Achmad Yurianto mengatakan, pemerintah mengkhawatirkan keberadaan orang tanpa gejala (OTG) yang menjadi sumber penularan Covid-19 di tengah masyarakat. "Kita masih tetap mewaspadai adanya penularan yang terjadi di lingkungan masyarakat. Ini disebabkan masih adanya kasus positif Covid-19 tanpa keluhan yang berada di tengah-tengah kita," ujar Yuri dalam konferensi pers di Graha BNPB, Sabtu (4/4/2020)

Artikel ini telah tayang di Kompas.com dengan judul "Jumlah Kasus Covid-19 Terus Bertambah, Pemerintah Khawatirkan Penularan dari OTG", https://nasional.kompas.com/read/2020/04/04/18382271/jumlah-kasus-covid-19-terus-bertambah-pemerintah-khawatirkan-penularan-dari.
Penulis : Dian Erika Nugraheny
Editor : Kristian Erdianto
"""

In [70]:
extract_keywords_tfidf(text3, tfidf)

['penularan',
 'keberadaan orang',
 'bertambah pemerintah',
 'masyarakat disebabkan',
 'orang gejala',
 'yuri',
 '`` mewaspadai',
 'lingkungan masyarakat',
 'pemerintah mengkhawatirkan',
 "tengah-tengah ''"]